<a href="https://colab.research.google.com/github/AnnaZhuravleva/compling/blob/master/assignment_1/Copy_of_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [0]:
import string
import re
import collections
from collections import Counter
import nltk
from sklearn.base import TransformerMixin
import numpy as np


In [4]:
from google.colab import drive
drive.mount('/content/drive')
text = open('/content/drive/My Drive/Colab Notebooks/compling/assignment_1/peace.txt')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:

text = text.read()
len(text)

3227581

In [6]:
def preprocess_text(text):
    punct = list(set(re.findall('\W', text)))
    punct.remove('.')
    words = ''
    for word in text.split():
        for letter in word:
            if letter not in punct:
                words += letter
        words += ' '
    text = words.lower()
        
    # replace all punctuation except dots with spaces
    # collapse multiple spaces into one '   ' -> ' '
    return text


text = preprocess_text(text)
len(text)
# assert len(text) == 3141169

3130570

In [0]:
text = text.split('.')
text = [x.strip() for x in text]

In [0]:
class BPE(TransformerMixin):
    
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        self.itos = list(set([item for txt in text for item in set(txt)]))
        self.stoi = {l: idx for idx, l in enumerate(self.itos)}
        text = [[self.stoi[letter] for letter in txt] for txt in text]
        
        while len(self.itos) < self.vocab_size:
            # TODO
            # count bigram freqencies in the text
            bigrams = collections.Counter()
            for txt in text:
                i = 0
                while i + 1 < len(txt):
                    bigrams[(txt[i], txt[i+1])] += 1
                    i += 1
                
                
            mc = bigrams.most_common(1)[0][0]
            new_token = str(self.itos[int(mc[0])]) + str(self.itos[int(mc[1])])
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            # find occurences of the new_token in the text and replace them with new_id
            tmp = []
            for txt in text:
                tmp2 = []
                i = 0
                while i + 1 < len(txt):
                    if new_token == self.itos[txt[i]] + self.itos[txt[i+1]]:
                        tmp2.append(new_id)
                        i += 2
                    else:
                        tmp2.append(txt[i])
                        i += 1
                tmp.append(tmp2)
                    
            text = tmp 
           
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """ 
        max_size = max([len(tok) for tok in self.itos])
        
        new_text = []
        for txt in text:
            i = 0
            new_txt = []
            while i < len(txt):
                hit = False
                stop = i+max_size if len(txt) - (i + max_size) >= 0 else len(txt)
                while hit == False and stop > i:
                    if txt[i:stop] in self.itos:
                        new_txt.append(self.stoi[txt[i:stop]])
                        hit = True
                        i = stop
                    else:
                        stop -= 1
            new_text.append(new_txt)
                                
        text = new_text
       # for token_id, token in enumerate(self.itos):
            # find occurences of the token in the text and replace them with token_id
          #  text = # TODO       
        return text
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        result = self.itos[tok] if isinstance(tok, int) else [self.itos[i] for i in token]
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [10]:
bpe.decode(tokenized_text[0]) 

'the project gutenberg ebook of war and peace by leo tolstoy this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever'

In [11]:
text[0]

'the project gutenberg ebook of war and peace by leo tolstoy this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever'

In [12]:
print(tokenized_text[0])

[65, 46, 52, 16, 47, 45, 0, 61, 13, 40, 31, 69, 30, 64, 73, 45, 30, 16, 16, 28, 20, 83, 34, 82, 20, 71, 46, 45, 35, 0, 57, 30, 66, 2, 45, 68, 31, 16, 2, 89, 16, 66, 58, 3, 60, 45, 30, 16, 16, 28, 20, 3, 60, 49, 75, 20, 65, 40, 23, 57, 83, 63, 12, 67, 57, 63, 12, 34, 9, 64, 57, 84, 48, 68, 0, 16, 89, 20, 71, 99, 58, 20, 87, 56, 16, 89, 20, 48, 68, 79, 89, 52, 3, 0, 31, 3, 67, 60, 34, 78, 31, 23, 16, 45, 17, 64]


In [0]:
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = Counter() # TODO create array for storing 3-gram counters
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = collections.defaultdict()
        for item in self.itos:
            result[item] = self.get_proba(a, b, item, tau)
        return result
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        # TODO approximate probability by counters
        
        P_abc = self.proba[a, b, c] + self.delta
        ab = [item for item in self.proba.keys() if item[0] == a \
              and item[1] == b]
        P_ab = sum([self.proba[item] for item in ab]) + self.delta
        n = 1/tau        
        result = (P_abc ** n) / (P_ab ** n) if P_abc > 0 else 0 
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        self.text = text
        self.itos = set([inner for sublist in text for inner in sublist])
        for txt in text:
            for i in range(len(txt) - 2):
                self.proba[txt[i], txt[i+1], txt[i+2]] += 1
                                       
        # self.proba = # TODO count 3-grams in the text
    
        return self
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    # TODO store in beam tuples of current sequences and their log probabilities
    
    if len(input_seq) >= 2:
        beam = [item for item in lm.proba.keys() if item[0] == input_seq[-2] \
                and item[1] == input_seq[-1]]
        beam = {item: lm.get_proba(item[0], item[1], item[2]) for item in beam}
    else:
        beam = [item for item in lm.proba.keys() if item[0] == input_seq[0]]
        beam = {item: lm.get_proba(item[0], item[1], item[2])  for item in beam}
        
    beam = sorted(beam.items(), key= lambda x: x[1], reverse=True)[:k]
    beam = {item[0] : item[1] for item in beam}
    
    for i in range(max_len):
        candidates = []
        candidates_proba = []
        for snt, snt_proba in beam.items():
            # if # TODO process terminal token нет нужной триграммы
            # else:    
            proba = lm.infer(snt[0], snt[1]) # probability vector of the next token
            best_k = sorted(proba.items(), key=lambda x: x[1], reverse=True)
            for k in best_k:
              candidates_proba.append(k[1] * snt_proba)
              key = list(snt)
              key.append(k[0])
              candidates.append(tuple(key))
        beam = {}
        for a in range(len(candidates)):
          beam[candidates[a]] = candidates_proba[a]
        beam = sorted(beam.items(), key=lambda x: x[1], reverse=True)[:5]
        beam = {i[0]: i[1] for i in beam}
        print (beam)            
    return beam

In [138]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
    

{(23, 57, 34, 34): 0.008537821395459212, (23, 57, 34, 83): 0.007022118271321509, (23, 57, 83, 34): 0.007022118271321509, (23, 57, 34, 71): 0.006273859767000364, (23, 57, 71, 34): 0.006273859767000364}
{(23, 57, 34, 34, 34): 0.0007888975334259446, (23, 57, 34, 34, 83): 0.0006488460611997656, (23, 57, 34, 83, 34): 0.0006488460611997656, (23, 57, 83, 34, 34): 0.0006488460611997656, (23, 57, 34, 34, 71): 0.0005797067268096266}
{(23, 57, 34, 34, 34, 34): 7.289439418076108e-05, (23, 57, 34, 34, 83, 34): 5.995359161833383e-05, (23, 57, 34, 83, 34, 34): 5.995359161833383e-05, (23, 57, 83, 34, 34, 34): 5.995359161833383e-05, (23, 57, 34, 34, 34, 83): 5.9953591618333825e-05}
{(23, 57, 34, 34, 34, 34, 34): 6.735466239709029e-06, (23, 57, 34, 34, 34, 34, 83): 5.539731783670796e-06, (23, 57, 34, 34, 83, 34, 34): 5.539731783670796e-06, (23, 57, 34, 83, 34, 34, 34): 5.539731783670796e-06, (23, 57, 83, 34, 34, 34, 34): 5.539731783670796e-06}
{(23, 57, 34, 34, 34, 34, 34, 34): 6.223593182455394e-07, (2

In [0]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

In [0]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities

In [0]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

In [0]:
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    result = #TODO perplexity for the sentence
    return result

perplexity(tokenized_text[0], lm)